# 00 Setup

- Adds `src/` files to the path
- Imports modules from `src/

In [0]:
import sys
import os

In [0]:
# Get current working directory (likely something like /Workspace/Users/username/spelling-bee-solver-training/notebooks)
cwd = os.getcwd()

# Move up one directory to get the repo root
repo_root = os.path.abspath(os.path.join(cwd, ".."))

# Construct the src path dynamically
src_path = os.path.join(repo_root, "src")

# Add to sys.path if not already there
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"Added {src_path} to sys.path")



In [0]:
# Now import modules
import constants

print("Setup complete: src/ added to sys.path and modules imported.")
